In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Shabbir\\Desktop\\YT_Prac\\Text-Summarization-Project\\research'

In [3]:
os.chdir("..")

In [4]:
%pwd

'c:\\Users\\Shabbir\\Desktop\\YT_Prac\\Text-Summarization-Project'

# Config.yaml and Entity:

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


# Configuration

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import create_directories, read_yaml
from pathlib import Path

class configurationManager:

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )  
          
        return data_transformation_config

# Components:

In [11]:
import os
from pathlib import Path
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.tokenizer_name
        )


    def convert_examples_to_features(self, example_batch):
        inputs_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length = 1024,
            truncation = True
        )

        with self.tokenizer.as_target_tokenizer():
            targets_encodings = self.tokenizer(
                example_batch['summary'],
                max_length = 128,
                truncation = True
            )

        return {
            'input_ids': inputs_encodings['input_ids'],
            'attention_mask': inputs_encodings['attention_mask'],
            'labels': targets_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features,
            batched = True
        )
        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, 'samsum_dataset')
        )

# Pipeline:

In [12]:
try:
    config = configurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2025-11-24 19:44:56,574: INFO: common]: YAML file: config\config.yaml loaded successfully
[2025-11-24 19:44:56,576: INFO: common]: YAML file: params.yaml loaded successfully
[2025-11-24 19:44:56,578: INFO: common]: Created directory at: artifacts
[2025-11-24 19:44:56,579: INFO: common]: Created directory at: artifacts/data_transformation


f:\conda_envs\textSum\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shabbir\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]f:\conda_envs\textSum\Lib\site-packages\transformers\tokenizati